In [1]:
# =============================================================================
# CELL 1: WINDOWS/VPN FIXES
# =============================================================================

import sys
import warnings

# Suppress noisy warnings
warnings.filterwarnings("ignore")

# === WINDOWS ASYNC FIX ===
if sys.platform == "win32":
    import asyncio

    try:
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
        print("Windows async policy set")
    except Exception as e:
        print(f"Async policy warning: {e}")

# === SSL/VPN FIXES ===
import os

os.environ["PYTHONHTTPSVERIFY"] = "0"
os.environ["CURL_CA_BUNDLE"] = ""

import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# === MAIN IMPORTS ===
import pandas as pd
import time

# === YAHOO IMPORTS (with error handling) ===
try:
    from yahooquery import Ticker

    print("yahooquery loaded")
except Exception as e:
    print(f"yahooquery failed: {e}")
    Ticker = None

try:
    import yfinance as yf

    print("yfinance loaded")
except Exception as e:
    print(f"yfinance failed: {e}")
    yf = None

from src.setup import *
from src.fetch import get_combined_universe

print("\nAll imports complete!")

✅ Windows async policy set
✅ yahooquery loaded
✅ yfinance loaded

🎉 All imports complete!


In [2]:
MIN_PRICE = 2.00  # Stock price must be at least $2 (avoid "penny stocks")
MIN_VOLUME = 1_000_000  # At least 1,000,000 shares traded per day (on average)

MIN_CAP = 300_000_000  # Market cap at least $300 million (company value)
# Market Cap = Stock Price × Number of Shares
MIN_CURRENT_RATIO = 1.2  # Current Ratio must be > 1.2
# Current Ratio = Current Assets / Current Liabilities
# If > 1, the company can pay its short-term bills
MAX_PE_RATIO = 100.0  # P/E ratio must be less than 100
# P/E = Price / Earnings Per Share


# 4. SAFETY THRESHOLDS - Additional quality filters
# -------------------------------------------------
MIN_INTEREST_COVERAGE = (
    1.5  # Interest Coverage > 1.5 means company earns 1.5x what it owes in interest
)
# Interest Coverage = EBIT / Interest Expense
# If < 1, company can't pay its interest bills = bad!
MIN_ROIC = 0.05  # ROIC (Return on Invested Capital) > 5%
# ROIC = Operating Profit / Capital Invested
# Shows how well management uses money to generate returns
FORTRESS_MARGIN_THRESHOLD = 0.05  # Operating Margin > 5% for "Fortress" (best) tier
# Operating Margin = Operating Income / Revenue
# Shows what % of sales becomes actual profit

# Sectors we want to EXCLUDE (skip)
# Financial Services and Real Estate have very different financial structures
EXCLUDED_SECTORS = ["Financial Services", "Real Estate"]

CACHE_EXPIRY_DAYS = 30  # Re-fetch data if our saved data is older than 30 days

In [3]:
from src.filters import get_initial_survivors
from src.financial_analysis import get_advanced_metrics

# STEP 1: Get list of all US stocks
tickers = get_combined_universe()

if len(tickers) > 0:
    # STEP 2: Apply quick filters
    survivors_df = get_initial_survivors(
        tickers,
        MIN_PRICE,
        MIN_VOLUME,
        MIN_CAP,
        MIN_CURRENT_RATIO,
        EXCLUDED_SECTORS,
        MAX_PE_RATIO,
    )

    if not survivors_df.empty:
        print(f"\n Step 2 Complete. {len(survivors_df)} stocks passed basic filters.")

        # STEP 3: Deep financial analysis
        final_results = get_advanced_metrics(survivors_df, CACHE_EXPIRY_DAYS,FORTRESS_MARGIN_THRESHOLD,MIN_INTEREST_COVERAGE,MIN_ROIC,calculate_altman_z_yfinance,save_cache)

        if not final_results.empty:
            # Sort results: First by Tier (alphabetically), then by Z-Score (highest first)
            final_results = final_results.sort_values(
                by=["Tier", "Z-Score"],
                ascending=[True, False],  # True = A-Z, False = highest first
            )

            # Split into three categories based on tier
            fortress_df = final_results[final_results["Tier"] == "Fortress"].copy()
            strong_df = final_results[final_results["Tier"] == "Strong"].copy()
            risky_df = final_results[final_results["Tier"] == "Risky"].copy()

            # Save to CSV files (spreadsheet format)
            try:
                fortress_df.to_csv(
                    FORTRESS_CSV, index=False
                )  # index=False means don't save row numbers
                strong_df.to_csv(STRONG_CSV, index=False)
                risky_df.to_csv(RISKY_CSV, index=False)

                print("\n" + "=" * 60)
                print("RESULTS GENERATED")
                print("=" * 60)
                print(f"1. FORTRESS ({len(fortress_df)}): Saved to '{FORTRESS_CSV}'")
                print(f"2. STRONG   ({len(strong_df)}): Saved to '{STRONG_CSV}'")
                print(f"3. RISKY    ({len(risky_df)}): Saved to '{RISKY_CSV}'")

            except Exception as e:
                print(f"\n  Error Saving Files: {e}")

            # Set display options for pandas (so we can see more data)
            pd.set_option("display.max_rows", 500)
            pd.set_option("display.max_columns", 20)
            pd.set_option("display.width", 1000)

            # Show preview of fortress stocks
            print("\n--- FORTRESS PREVIEW ---")
            print(fortress_df.head(15))  # .head(15) shows first 15 rows
        else:
            print("No stocks passed the deep financial analysis.")
    else:
        print("No stocks passed the initial lightweight filter.")
else:
    print("Could not fetch ticker universe.")

--- STEP 1: Fetching North American Universe ---
   -> Found 6008 US stocks.

--- STEP 2: Running 'Lightweight' Filter on 6008 stocks ---
 -> Processing Batch 1/13...
 -> Processing Batch 6/13...
 -> Processing Batch 11/13...

 Step 2 Complete. 554 stocks passed basic filters.

--- STEP 3: Fetching Deep Financials for 554 Survivors ---
 -> Analyzing 1/554: A...
 -> Analyzing 21/554: ALGN...
 -> Analyzing 41/554: AQN...
 -> Analyzing 61/554: AXL...
 -> Analyzing 81/554: BMY...
 -> Analyzing 101/554: CCC...
 -> Analyzing 121/554: CNXC...
 -> Analyzing 141/554: CXM...
 -> Analyzing 161/554: DT...
 -> Analyzing 181/554: ETN...
 -> Analyzing 201/554: FOLD...
 -> Analyzing 221/554: GIL...
 -> Analyzing 241/554: HALO...
 -> Analyzing 261/554: HSY...
 -> Analyzing 281/554: ITGR...
 -> Analyzing 301/554: LFST...
 -> Analyzing 321/554: MHK...
 -> Analyzing 341/554: NFLX...
 -> Analyzing 361/554: OLLI...
 -> Analyzing 381/554: PGY...
 -> Analyzing 401/554: QXO...
 -> Analyzing 421/554: RVTY...
 -

In [4]:
# =============================================================================
# CELL 6a: FORTRESS PULLBACK SCAN (RSI + TREND)
# =============================================================================
# This scan looks for high-quality "Fortress" stocks that are:
# 1. In a long-term Uptrend (Price > 200 SMA)
# 2. In a short-term Pullback (RSI < 40)
# This is the "Buy the Dip" strategy for quality assets.

import yfinance as yf
import pandas as pd


def apply_fortress_pullback(df_input, rsi_threshold=40):
    """
    Calculates RSI (14-day) and 200-day SMA.
    Filters for stocks where Price > SMA200 AND RSI < Threshold.
    """
    if df_input is None or df_input.empty:
        print("   [Fortress Pullback] Input DataFrame is empty. Skipping.")
        return pd.DataFrame()

    tickers = df_input["Ticker"].tolist()
    print(
        f"\n--- FORTRESS PULLBACK: Scanning {len(tickers)} stocks (RSI < {rsi_threshold}) ---"
    )

    # 1. Fetch History (Need ~1 year for SMA 200)
    try:
        data = yf.download(
            tickers,
            period="1y",
            interval="1d",
            progress=False,
            group_by="ticker",
            auto_adjust=True,
        )
    except Exception as e:
        print(f"   [Error] Failed to fetch history: {e}")
        return pd.DataFrame()

    pullback_candidates = []

    for ticker in tickers:
        try:
            # Handle Data Structure
            if len(tickers) > 1:
                if ticker not in data.columns.levels[0]:
                    continue
                df_hist = data[ticker].copy()
            else:
                df_hist = data.copy()

            # Need enough data
            if len(df_hist) < 200:
                print(f"   [Skipping] {ticker}: Insufficient history.")
                continue

            # 2. Calculate 200-Day SMA (Trend Filter)
            sma_200 = df_hist["Close"].rolling(window=200).mean().iloc[-1]
            current_price = df_hist["Close"].iloc[-1]

            # CONDITION 1: Must be in an Uptrend
            if current_price < sma_200:
                continue

            # 3. Calculate RSI (14-Day)
            # Calculate daily price changes
            delta = df_hist["Close"].diff()

            # Separate gains and losses
            gain = delta.where(delta > 0, 0)
            loss = -delta.where(delta < 0, 0)

            # Calculate Exponential Moving Average (Wilder's Smoothing usually uses alpha=1/14)
            avg_gain = gain.ewm(com=13, adjust=False).mean()
            avg_loss = loss.ewm(com=13, adjust=False).mean()

            # Calculate RS and RSI
            rs = avg_gain / avg_loss
            rsi = 100 - (100 / (1 + rs))

            current_rsi = rsi.iloc[-1]

            # CONDITION 2: Must be Oversold (Pullback)
            if current_rsi < rsi_threshold:
                # Get base data
                base_row = df_input[df_input["Ticker"] == ticker].iloc[0].to_dict()

                base_row["RSI_14"] = round(current_rsi, 2)
                base_row["SMA_200"] = round(sma_200, 2)
                # Distance from SMA (Buy Zone Depth)
                base_row["Dist_to_SMA_%"] = round(
                    ((current_price - sma_200) / sma_200) * 100, 2
                )

                pullback_candidates.append(base_row)

        except Exception as e:
            continue

    return pd.DataFrame(pullback_candidates)


# =========================================
# EXECUTION
# =========================================

# Check if Fortress DataFrame exists
if "fortress_df" in locals() and not fortress_df.empty:

    # Run the scan
    # We use 40 as the threshold because quality stocks rarely drop to 30
    Fortress_Dip_DF = apply_fortress_pullback(fortress_df, rsi_threshold=40)

    if not Fortress_Dip_DF.empty:
        # Sort by Lowest RSI (Deepest Dip)
        Fortress_Dip_DF = Fortress_Dip_DF.sort_values(by="RSI_14", ascending=True)

        # Save to CSV
        PULLBACK_CSV = "YfinanceDataDump/Fortress_Pullback_Candidates_USA.csv"
        Fortress_Dip_DF.to_csv(PULLBACK_CSV, index=False)

        print(f"\nSUCCESS: Pullback candidates saved to '{PULLBACK_CSV}'")
        print(
            f"Original Fortress Count: {len(fortress_df)} -> Pullback Count: {len(Fortress_Dip_DF)}"
        )

        cols = ["Ticker", "Price", "RSI_14", "SMA_200", "Dist_to_SMA_%", "Tier"]
        print("\n--- FORTRESS STOCKS 'ON SALE' ---")
        print(Fortress_Dip_DF[cols].head(15))
    else:
        print(
            "\nRESULT: 0 stocks passed. No Fortress stocks are currently oversold (RSI < 40)."
        )
else:
    print("Error: 'fortress_df' not found. Please run the Main Execution cell first.")


--- FORTRESS PULLBACK: Scanning 220 stocks (RSI < 40) ---

SUCCESS: Pullback candidates saved to 'YfinanceDataDump/Fortress_Pullback_Candidates_USA.csv'
Original Fortress Count: 220 -> Pullback Count: 10

--- FORTRESS STOCKS 'ON SALE' ---
  Ticker    Price  RSI_14  SMA_200  Dist_to_SMA_%      Tier
7   WLTH    9.500   14.38      NaN            NaN  Fortress
8   VSNT   33.040   22.74      NaN            NaN  Fortress
5   QCOM  159.420   32.30   158.62           0.50  Fortress
0   ISRG  535.068   34.70   514.74           3.94  Fortress
3   NBIX  132.390   35.37   132.19           0.15  Fortress
4    ULS   75.000   35.38    71.79           4.47  Fortress
1    APP  568.760   35.68   482.72          17.82  Fortress
9   TGNA   18.850   36.52    18.35           2.73  Fortress
2     ZM   80.960   36.79    80.14           1.03  Fortress
6   URBN   69.480   38.30    68.67           1.17  Fortress


In [22]:
# =============================================================================
# CELL 7: STEP 4 - ANALYST RATINGS FILTER
# =============================================================================
# This adds another layer of analysis: what do professional analysts think?
# Analyst Rating Scale (typically):
#   1.0 = Strong Buy
#   2.0 = Buy
#   3.0 = Hold
#   4.0 = Sell
#   5.0 = Strong Sell


def get_analyst_fortress_from_var(df_input):
    """
    Fetch analyst ratings for stocks and filter for those rated "Buy" or better.

    Args:
        df_input: DataFrame of stocks to analyze (usually fortress_df)

    Returns:
        DataFrame: Stocks with analyst ratings and upside potential
    """
    working_df = df_input.copy()
    tickers = working_df["Ticker"].tolist()

    print(f"\n--- STEP 4: Fetching Analyst Ratings for {len(tickers)} Stocks ---")
    print("    (Fetching serially to avoid throttling...)")

    analyst_data = []

    for i, ticker in enumerate(tickers):
        # Print progress every 10 stocks
        if i % 10 == 0:
            print(f" -> Analyst Scan {i+1}/{len(tickers)}: {ticker}...")

        try:
            stock = yf.Ticker(ticker)
            info = stock.info  # Get all available info for the stock

            # Extract analyst recommendation (1.0 to 5.0 scale)
            rec_mean = info.get("recommendationMean")
            target_price = info.get("targetMeanPrice")  # Average price target
            current_price = info.get("currentPrice")

            # Filter: Only keep stocks rated 2.5 or better (Buy or Strong Buy)
            if rec_mean is None or rec_mean > 2.5:
                continue

            # Calculate upside potential
            # Upside = (Target - Current) / Current * 100
            upside = 0
            if target_price and current_price:
                upside = round(
                    ((target_price - current_price) / current_price) * 100, 2
                )

            # Get the original data and add new columns
            base_row = working_df[working_df["Ticker"] == ticker].iloc[0].to_dict()
            base_row["Analyst_Rating"] = rec_mean
            base_row["Target_Price"] = target_price
            base_row["Upside_%"] = upside

            analyst_data.append(base_row)
            time.sleep(0.2)  # Small delay to not be timed out of Yahoo's servers

        except Exception:
            print(f"Error fetching {ticker}: {e}")
            continue

    return pd.DataFrame(analyst_data)


# ==========================================
# EXECUTE THE ANALYST FILTER
# ==========================================

# Check if fortress_df exists from the previous step
if "fortress_df" in locals() and not fortress_df.empty:

    # Run the function
    Analyst_Fortress_DF = get_analyst_fortress_from_var(fortress_df)

    if not Analyst_Fortress_DF.empty:
        # Sort by highest upside potential
        Analyst_Fortress_DF = Analyst_Fortress_DF.sort_values(
            by="Upside_%", ascending=False
        )

        print("\n  Analyst Scan Complete!")
        print(f"Found {len(Analyst_Fortress_DF)} stocks with Buy Ratings (Score < 2.5)")

        # Save to CSV
        Analyst_Fortress_DF.to_csv(ANALYST_CSV, index=False)
        print(f"Saved to '{ANALYST_CSV}'")

        # Show top picks
        cols = ["Ticker", "Price", "Analyst_Rating", "Upside_%", "Target_Price", "Tier"]
        print(Analyst_Fortress_DF[cols].head())
    else:
        print("No stocks passed the Analyst filter.")
else:
    print("  'fortress_df' not found. Please run Steps 1-3 first.")


--- STEP 4: Fetching Analyst Ratings for 220 Stocks ---
    (Fetching serially to avoid throttling...)
 -> Analyst Scan 1/220: WPM...
 -> Analyst Scan 11/220: ANET...
 -> Analyst Scan 21/220: CPRX...
 -> Analyst Scan 31/220: META...
 -> Analyst Scan 41/220: LLY...
 -> Analyst Scan 51/220: TXN...
 -> Analyst Scan 61/220: REGN...
 -> Analyst Scan 71/220: AEM...
 -> Analyst Scan 81/220: ULS...
 -> Analyst Scan 91/220: PAAS...
 -> Analyst Scan 101/220: ABT...
 -> Analyst Scan 111/220: ECL...
 -> Analyst Scan 121/220: LRN...
 -> Analyst Scan 131/220: ATI...
 -> Analyst Scan 141/220: DKS...
 -> Analyst Scan 151/220: GNRC...
 -> Analyst Scan 161/220: TOL...
 -> Analyst Scan 171/220: TMHC...
 -> Analyst Scan 181/220: TPR...
 -> Analyst Scan 191/220: BLDR...
 -> Analyst Scan 201/220: BURL...
 -> Analyst Scan 211/220: BIRK...

  Analyst Scan Complete!
Found 186 stocks with Buy Ratings (Score < 2.5)
Saved to 'YfinanceDataDump\Analyst_Fortress_Picks.csv'
    Ticker    Price  Analyst_Rating  Upsid

In [24]:
# =============================================================================
# CELL 8: BUFFETT VALUE SCAN
# =============================================================================
# This filter looks for stocks Warren Buffett might like:
# - Trading BELOW book value (P/B < 1.0) means you're buying $1 of assets for less than $1
# - Positive Return on Equity (ROE) shows the company is profitable
# - Reasonable debt levels (Debt/Equity < 100%)


def get_buffett_value_picks(df_input):
    """
    Find deep value stocks trading below their book value.

    Book Value = Total Assets - Total Liabilities
    P/B Ratio = Stock Price / Book Value per Share

    If P/B < 1.0, you're theoretically buying the company for less than
    what it would be worth if you sold all its assets and paid all debts.

    Args:
        df_input: DataFrame of stocks to analyze

    Returns:
        DataFrame: Deep value stocks sorted by P/B ratio
    """
    print(f"\n--- STEP 5: Warren Buffett 'Below NAV' Scan ---")
    print(f"    Scanning {len(df_input)} candidates for Deep Value...")
    print(
        "    Criteria: P/B < 1.0 (Below Book) | ROE > 0% (Profitable) | Debt/Eq < 100%"
    )

    tickers = df_input["Ticker"].tolist()
    buffett_candidates = []

    # Process in chunks for speed
    chunk_size = 250
    chunks = [tickers[i : i + chunk_size] for i in range(0, len(tickers), chunk_size)]

    for chunk in chunks:
        try:
            yq = Ticker(chunk, asynchronous=True)
            # Get key statistics and financial data
            data = yq.get_modules("defaultKeyStatistics financialData")

            for symbol in chunk:
                if isinstance(data, dict) and symbol in data:
                    try:
                        stats = data[symbol].get("defaultKeyStatistics", {})
                        fin = data[symbol].get("financialData", {})

                        # 1. Price to Book < 1.0 (The Core "Value" Rule)
                        pb = stats.get("priceToBook")
                        if pb is None or pb >= 1.0 or pb <= 0:
                            continue

                        # 2. Positive ROE (Return on Equity - company makes money)
                        roe = fin.get("returnOnEquity", 0)
                        if roe is None or roe <= 0:
                            continue

                        # 3. Reasonable Debt (not overleveraged)
                        de = fin.get("debtToEquity", 0)
                        if de is None or de > 100:
                            continue

                        # Get base data and add value metrics
                        base_row = (
                            df_input[df_input["Ticker"] == symbol].iloc[0].to_dict()
                        )
                        base_row["P/B Ratio"] = round(pb, 2)
                        base_row["ROE %"] = round(roe * 100, 2)
                        base_row["Debt/Eq %"] = round(de, 2)

                        buffett_candidates.append(base_row)

                    except:
                        print(f"   [Error] Processing {symbol} data.")
                        continue
        except:
            print("   [Error] Fetching chunk data.")
            continue

    return pd.DataFrame(buffett_candidates)


# Run the Buffett scan
if "final_results" in locals() and not final_results.empty:
    Buffett_Value_DF = get_buffett_value_picks(final_results)

    if not Buffett_Value_DF.empty:
        Buffett_Value_DF = Buffett_Value_DF.sort_values(by="P/B Ratio", ascending=True)
        Buffett_Value_DF.to_csv(BUFFETT_CSV, index=False)

        print("\n" + "=" * 60)
        print("BUFFETT SCAN COMPLETE")
        print("=" * 60)
        print(f"Found {len(Buffett_Value_DF)} Deep Value Stocks")

        cols = ["Ticker", "Price", "P/B Ratio", "ROE %", "Debt/Eq %", "Sector", "Tier"]
        print("\n--- DEEP VALUE PICKS ---")
        print(Buffett_Value_DF[cols].head())
    else:
        print("\n  No stocks passed the Buffett filter.")
else:
    print(" 'final_results' not found. Please run Steps 1-3 first.")


--- STEP 5: Warren Buffett 'Below NAV' Scan ---
    Scanning 550 candidates for Deep Value...
    Criteria: P/B < 1.0 (Below Book) | ROE > 0% (Profitable) | Debt/Eq < 100%

BUFFETT SCAN COMPLETE
Found 18 Deep Value Stocks

--- DEEP VALUE PICKS ---
  Ticker  Price  P/B Ratio  ROE %  Debt/Eq %                  Sector      Tier
9   ACHC  11.68       0.34   3.69      74.49              Healthcare     Risky
0   GMAB  32.25       0.35  29.41       2.47              Healthcare  Fortress
8     KT  19.49       0.40   5.08      60.17  Communication Services     Risky
5    SSL   6.31       0.44   5.07      76.56         Basic Materials     Risky
7   ANGI  12.49       0.55   3.42      54.03  Communication Services     Risky


In [25]:
# =============================================================================
# CELL 8a: BUFFETT TREND ALIGNMENT (200-DAY SMA)
# =============================================================================
# This cell takes the 'Deep Value' candidates found in the previous step and
# filters out any that are in a long-term downtrend (Price < 200-Day SMA).
# This helps avoid "Value Traps" - stocks that are cheap because they are dying.

import yfinance as yf
import pandas as pd


def apply_trend_alignment(df_input):
    """
    Calculates the 200-day SMA and filters for stocks trading ABOVE it.
    """
    if df_input is None or df_input.empty:
        print("   [Trend Alignment] Input DataFrame is empty. Skipping.")
        return pd.DataFrame()

    tickers = df_input["Ticker"].tolist()
    print(
        f"\n--- TREND ALIGNMENT: Checking 200-Day SMA for {len(tickers)} Buffett candidates ---"
    )

    # 1. Fetch History (Need at least 200 trading days, so 1y is perfect)
    try:
        # group_by='ticker' ensures the dataframe structure is consistent
        data = yf.download(
            tickers,
            period="1y",
            interval="1d",
            progress=False,
            group_by="ticker",
            auto_adjust=True,
        )
    except Exception as e:
        print(f"   [Error] Failed to fetch history: {e}")
        return pd.DataFrame()

    trend_data = []

    for ticker in tickers:
        try:
            # Handle data structure: if multiple tickers, it uses MultiIndex
            if len(tickers) > 1:
                if ticker not in data.columns.levels[0]:
                    continue
                df_hist = data[ticker].copy()
            else:
                # If only one ticker, yfinance returns a flat dataframe
                df_hist = data.copy()

            # Need at least 200 days of data to calculate the SMA
            if len(df_hist) < 200:
                print(f"   [Skipping] {ticker}: Insufficient history (<200 days).")
                continue

            # 2. Calculate 200-Day SMA
            # .rolling(window=200).mean() averages the last 200 closing prices
            sma_200 = df_hist["Close"].rolling(window=200).mean().iloc[-1]
            current_price = df_hist["Close"].iloc[-1]

            # 3. The Filter Condition: Price > SMA 200
            # We want stocks that have "reclaimed" their trend
            is_uptrend = current_price > sma_200

            # Calculate how far above/below the SMA it is (as a %)
            distance_pct = round(((current_price - sma_200) / sma_200) * 100, 2)

            if is_uptrend:
                # Get the original row data from the Buffett DataFrame
                base_row = df_input[df_input["Ticker"] == ticker].iloc[0].to_dict()

                # Add technical metrics to the row
                base_row["SMA_200"] = round(sma_200, 2)
                base_row["Trend_Dist_%"] = distance_pct

                trend_data.append(base_row)

        except Exception as e:
            print(f"   [Error] Processing {ticker}: {e}")
            continue

    # Create the new filtered DataFrame
    return pd.DataFrame(trend_data)


# =========================================
# EXECUTION
# =========================================

# Check if the Buffett DataFrame exists from the previous cell
if "Buffett_Value_DF" in locals() and not Buffett_Value_DF.empty:

    # Run the filter
    Buffett_Trend_DF = apply_trend_alignment(Buffett_Value_DF)

    if not Buffett_Trend_DF.empty:
        # Sort by trend strength (how far above the 200 SMA they are)
        Buffett_Trend_DF = Buffett_Trend_DF.sort_values(
            by="Trend_Dist_%", ascending=True
        )

        # Save to a new CSV
        TREND_CSV = "YfinanceDataDump/Buffett_Trend_Aligned_USA.csv"
        Buffett_Trend_DF.to_csv(TREND_CSV, index=False)

        print(f"\nSUCCESS: Filtered list saved to '{TREND_CSV}'")
        print(
            f"Original Count: {len(Buffett_Value_DF)} -> Trend Aligned Count: {len(Buffett_Trend_DF)}"
        )

        # Display the survivors
        cols = ["Ticker", "Price", "P/B Ratio", "SMA_200", "Trend_Dist_%", "Tier"]
        print("\n--- BUFFETT STOCKS IN UPTREND ---")
        print(Buffett_Trend_DF[cols].head(5))
    else:
        print(
            "\nRESULT: 0 stocks passed. All Buffett picks are currently in a downtrend."
        )
else:
    print(
        "Error: 'Buffett_Value_DF' not found. Please run the Buffett Scan cell first."
    )


--- TREND ALIGNMENT: Checking 200-Day SMA for 18 Buffett candidates ---

SUCCESS: Filtered list saved to 'YfinanceDataDump/Buffett_Trend_Aligned_USA.csv'
Original Count: 18 -> Trend Aligned Count: 15

--- BUFFETT STOCKS IN UPTREND ---
   Ticker  Price  P/B Ratio  SMA_200  Trend_Dist_%      Tier
1      KT  19.49       0.40    19.26          1.17     Risky
11    SXC   7.91       0.96     7.79          1.59    Strong
14   TMHC  63.27       1.00    61.74          2.47  Fortress
12   TGNA  18.85       0.97    18.35          2.73  Fortress
13    KBH  61.32       0.99    58.20          5.37     Risky


In [28]:
# =============================================================================
# CELL 9: INSIDER TRADING FILTER
# =============================================================================
# Insiders (executives, directors, major shareholders) sometimes have better
# information about their company. When they BUY their own stock, it can be
# a positive sign. When they SELL, it might be neutral (paying taxes) or negative.

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)


def filter_for_insider_buying(tickers):
    """
    Find stocks where company insiders are NET BUYERS.

    Net Buying = Total shares bought - Total shares sold
    If positive, insiders are accumulating shares (bullish signal).

    Args:
        tickers: List of stock symbols to check

    Returns:
        DataFrame: Stocks with positive insider buying
    """
    print(f" Scanning {len(tickers)} stocks for Insider Buying...")
    insider_picks = []

    # Process in smaller chunks to avoid timeout
    chunk_size = 25
    chunks = [tickers[i : i + chunk_size] for i in range(0, len(tickers), chunk_size)]

    for chunk in chunks:
        try:
            yq = Ticker(chunk, asynchronous=True)

            # Fetch insider transaction data
            df_insiders = yq.insider_transactions

            # Fetch current prices
            price_data = yq.price

            # Skip if no data
            if isinstance(df_insiders, dict) or not hasattr(df_insiders, "reset_index"):
                continue

            df_insiders = df_insiders.reset_index()

            for symbol in chunk:
                if symbol not in df_insiders["symbol"].values:
                    continue

                # Get transactions for this stock
                stock_tx = df_insiders[df_insiders["symbol"] == symbol].copy()

                # Separate purchases and sales
                # The text typically says "Purchase" or "Sale"
                buys = stock_tx[
                    stock_tx["transactionText"]
                    .astype(str)
                    .str.contains("Purchase", case=False, na=False)
                ]
                sells = stock_tx[
                    stock_tx["transactionText"]
                    .astype(str)
                    .str.contains("Sale", case=False, na=False)
                ]

                # Calculate total buy/sell volume
                buy_vol = buys["shares"].sum() if not buys.empty else 0
                sell_vol = sells["shares"].sum() if not sells.empty else 0

                # Get current price
                current_price = None
                try:
                    if isinstance(price_data, dict) and symbol in price_data:
                        current_price = price_data[symbol].get(
                            "regularMarketPrice", None
                        )
                except:
                    current_price = None

                # Only keep if net buying is positive
                if buy_vol > sell_vol:
                    insider_picks.append(
                        {
                            "Ticker": symbol,
                            "Current_Price": current_price,
                            "Insider_Buys_Count": len(buys),
                            "Net_Shares_Bought": buy_vol - sell_vol,
                        }
                    )

        except Exception as e:
            print(f"   [Error] Processing chunk: {e}")
            continue

    return pd.DataFrame(insider_picks)


# Run the insider filter
if "fortress_df" in locals() and not fortress_df.empty:
    target_tickers = fortress_df["Ticker"].tolist()

else:
    print("'fortress_df' and 'strong_df' not found.")

Fortress_insiders_1 = filter_for_insider_buying(target_tickers)
print(f"Created 'Fortress_insiders' with {len(Fortress_insiders_1)} rows.")
Fortress_insiders_1

 Scanning 220 stocks for Insider Buying...
Created 'Fortress_insiders' with 12 rows.


,Ticker,Current_Price,Insider_Buys_Count,Net_Shares_Bought
0,MU,362.750,3,167433.0
1,CDE,22.580,21,1650770.0
2,HXL,82.590,6,14327.0
3,GIL,63.220,105,1426104.0
4,SHLS,9.270,3,8335.0
5,CGAU,16.130,87,4297682.0
6,RAL,53.530,1,2000.0
7,TTD,35.477,10,3557415.0
8,FSM,10.420,10,2116207.0
9,OPCH,36.030,8,18954.0


In [27]:
# =============================================================================
# CELL 9a: INSIDER TRADING FILTER (STRONG LIST ONLY | MIN 2 BUYS)
# =============================================================================
import warnings
import pandas as pd
import time
from yahooquery import Ticker

warnings.simplefilter(action="ignore", category=FutureWarning)


def filter_for_strong_insider_buying(tickers):
    """
    Find stocks where insiders are NET BUYERS with HIGH CONVICTION.
    Criteria:
    1. Net Buying (Buys > Sells)
    2. Conviction (At least 2 separate Buy transactions)
    """
    print(f" Scanning {len(tickers)} stocks for High-Conviction Insider Buying...")
    insider_picks = []

    # Conservative chunk size for stability
    chunk_size = 15
    chunks = [tickers[i : i + chunk_size] for i in range(0, len(tickers), chunk_size)]

    for i, chunk in enumerate(chunks):
        try:
            # Delay to prevent API blocking 
            #potentially add a while loop to iterate when a symbol is blocked rety in a couple seconds
            time.sleep(5.0)

            yq = Ticker(chunk, asynchronous=True)
            df_insiders = yq.insider_transactions
            price_data = yq.price

            if isinstance(df_insiders, dict) or not hasattr(df_insiders, "reset_index"):
                continue

            df_insiders = df_insiders.reset_index()

            for symbol in chunk:
                if symbol not in df_insiders["symbol"].values:
                    continue

                stock_tx = df_insiders[df_insiders["symbol"] == symbol].copy()

                # Robust lowercase search
                buys = stock_tx[
                    stock_tx["transactionText"]
                    .astype(str)
                    .str.lower()
                    .str.contains("purchase", na=False)
                ]
                sells = stock_tx[
                    stock_tx["transactionText"]
                    .astype(str)
                    .str.lower()
                    .str.contains("sale", na=False)
                ]

                # === NEW FILTER: MINIMUM 2 BUY TRANSACTIONS ===
                if len(buys) < 2:
                    continue

                buy_vol = buys["shares"].sum() if not buys.empty else 0
                sell_vol = sells["shares"].sum() if not sells.empty else 0

                # Logic: Net Positive Buying
                if buy_vol > sell_vol:
                    current_price = None
                    try:
                        if isinstance(price_data, dict) and symbol in price_data:
                            current_price = price_data[symbol].get(
                                "regularMarketPrice", None
                            )
                    except:
                        current_price = 0

                    insider_picks.append(
                        {
                            "Ticker": symbol,
                            "Current_Price": current_price,
                            "Insider_Buys_Count": len(buys),
                            "Net_Shares_Bought": buy_vol - sell_vol,
                        }
                    )

        except Exception as e:
            print(f"   [Error] Processing chunk {i+1}: {e}")
            continue

    return pd.DataFrame(insider_picks)


# =========================================
# EXECUTION (STRONG_DF ONLY)
# =========================================

target_tickers = []

# Check for 'strong_df' (or variations of the name)
if "strong_df" in locals() and not strong_df.empty:
    print(f"Source: 'strong_df' ({len(strong_df)} tickers)")
    target_tickers = strong_df["Ticker"].tolist()

else:
    print(
        "Error: 'strong_df' variable not found. Please run the Strong Filter cell first."
    )

# Run Scan
if len(target_tickers) > 0:
    # Remove duplicates and Sort
    unique_tickers = sorted(list(set(target_tickers)))

    Strong_insiders = filter_for_strong_insider_buying(unique_tickers)

    print(
        f"\nFinal Result: Found {len(Strong_insiders)} Strong stocks with 2+ Insider Buys."
    )

    if not Strong_insiders.empty:
        # Sort by Conviction
        Strong_insiders = Strong_insiders.sort_values(
            by="Net_Shares_Bought", ascending=False
        )
        Strong_insiders.head()

Source: 'strong_df' (107 tickers)
 Scanning 107 stocks for High-Conviction Insider Buying...
   [Error] Processing chunk 8: 'symbol'

Final Result: Found 12 Strong stocks with 2+ Insider Buys.


In [10]:
Fortress_insiders = pd.concat([Fortress_insiders_1, Strong_insiders], ignore_index=True)
Fortress_insiders.head()

,Ticker,Current_Price,Insider_Buys_Count,Net_Shares_Bought
0,MU,362.75,3,167433.0
1,CDE,22.58,21,1650770.0
2,HXL,82.59,6,14327.0
3,GIL,63.22,105,1426104.0
4,SHLS,9.27,3,8335.0


In [11]:
# =============================================================================
# CELL 9b: RELATIVE VOLUME (RVOL) CATALYST SCAN
# =============================================================================
# This checks your "Insider Picks" for unusual institutional activity.
# RVOL > 1.5 means volume is 150% of normal (institutions are active).
# RVOL > 2.0 is a massive "Ignition" signal.

import yfinance as yf
import pandas as pd


def apply_rvol_catalyst(df_input, rvol_threshold=1.5):
    """
    Calculates Relative Volume (RVOL).
    RVOL = Current Volume / Average Volume (30-day).
    """
    if df_input is None or df_input.empty:
        print("   [RVOL Catalyst] Input DataFrame is empty. Skipping.")
        return pd.DataFrame()

    tickers = df_input["Ticker"].tolist()
    print(f"\n--- RVOL CATALYST: Checking volume for {len(tickers)} Insider picks ---")

    # 1. Fetch History (Need ~2 months to get a clean 30-day average)
    try:
        data = yf.download(
            tickers,
            period="3mo",
            interval="1d",
            progress=False,
            group_by="ticker",
            auto_adjust=True,
        )
    except Exception as e:
        print(f"   [Error] Failed to fetch history: {e}")
        return pd.DataFrame()

    catalyst_data = []

    for ticker in tickers:
        try:
            # Handle data structure
            if len(tickers) > 1:
                if ticker not in data.columns.levels[0]:
                    continue
                df_hist = data[ticker].copy()
            else:
                df_hist = data.copy()

            # Need at least 30 days of volume data
            if len(df_hist) < 30:
                continue

            # 2. Get Volumes
            # Current Volume (Last completed bar)
            current_vol = df_hist["Volume"].iloc[-1]

            # Average Volume (Previous 30 days, excluding today to avoid skewing the average)
            avg_vol = df_hist["Volume"].iloc[-31:-1].mean()

            # Avoid division by zero
            if avg_vol == 0:
                continue

            # 3. Calculate RVOL
            rvol = current_vol / avg_vol

            # 4. Filter
            if rvol > rvol_threshold:
                # Get base data
                base_row = df_input[df_input["Ticker"] == ticker].iloc[0].to_dict()

                base_row["RVOL"] = round(rvol, 2)
                base_row["Avg_Vol_30D"] = int(avg_vol)
                base_row["Current_Vol"] = int(current_vol)

                catalyst_data.append(base_row)

        except Exception as e:
            print(f"   [Error] Processing {ticker}: {e}")
            continue

    return pd.DataFrame(catalyst_data)


# =========================================
# EXECUTION
# =========================================

# We check 'Fortress_insiders' first (Best List)
# If that doesn't exist, we check the raw 'Fortress_insiders' list
if "Fortress_insiders" in locals() and not Fortress_insiders.empty:
    target_df = Fortress_insiders
    print("Source: Analyst-Vetted Insider List")
elif "Fortress_insiders" in locals() and not Fortress_insiders.empty:
    target_df = Fortress_insiders
    print("Source: Raw Insider List")
else:
    target_df = pd.DataFrame()

if not target_df.empty:
    # Run the scan
    # Threshold 1.5 = 150% of normal volume
    Insider_Catalyst_DF = apply_rvol_catalyst(target_df, rvol_threshold=1.5)

    if not Insider_Catalyst_DF.empty:
        # Sort by highest RVOL (Biggest Catalyst)
        Insider_Catalyst_DF = Insider_Catalyst_DF.sort_values(
            by="RVOL", ascending=False
        )

        # Save results
        CATALYST_CSV = "YfinanceDataDump/Insider_RVOL_Catalysts_USA.csv"
        Insider_Catalyst_DF.to_csv(CATALYST_CSV, index=False)

        print(f"\nSUCCESS: Catalyst candidates saved to '{CATALYST_CSV}'")
        print(
            f"Original Insider Count: {len(target_df)} -> Catalyst Count: {len(Insider_Catalyst_DF)}"
        )

        # --- FIX IS HERE: Changed 'Price' to 'Current_Price' ---
        cols = [
            "Ticker",
            "Current_Price",
            "RVOL",
            "Insider_Buys_Count",
            "Net_Shares_Bought",
        ]

        # Only add Analyst_Verdict if it actually exists in the dataframe
        if "Analyst_Verdict" in Insider_Catalyst_DF.columns:
            cols.append("Analyst_Verdict")

        print("\n--- INSIDER STOCKS WITH HIGH VOLUME SPIKES ---")
        print(Insider_Catalyst_DF[cols].head(15))
    else:
        print(
            "\nRESULT: 0 stocks passed. No Insider picks are currently seeing high relative volume (>1.5x)."
        )
else:
    print("Error: No Insider DataFrame found. Please run the Insider Scan cell first.")

Source: Analyst-Vetted Insider List

--- RVOL CATALYST: Checking volume for 24 Insider picks ---

SUCCESS: Catalyst candidates saved to 'YfinanceDataDump/Insider_RVOL_Catalysts_USA.csv'
Original Insider Count: 24 -> Catalyst Count: 5

--- INSIDER STOCKS WITH HIGH VOLUME SPIKES ---
  Ticker  Current_Price  RVOL  Insider_Buys_Count  Net_Shares_Bought
4   AXTA          33.57  1.97                   7             7366.0
0     MU         362.75  1.66                   3           167433.0
2   SHLS           9.27  1.64                   3             8335.0
3   HSIC          79.98  1.59                   5            16257.0
1    CDE          22.58  1.50                  21          1650770.0


In [12]:
# ==========================================
# 10. ANALYST FILTER FUNCTION FOR INSIDER PICKS (NEW)
# ==========================================
def filter_for_analyst_ratings(df_insiders, max_score=2.5):
    """
    Fetches analyst data for the insider winners and filters for 'Buy' or better.
    Scale: 1.0 = Strong Buy, 5.0 = Sell.
    Cutoff: 2.5 ensures we get 'Buy' and 'Strong Buy'.
    """
    if df_insiders.empty:
        return df_insiders

    tickers = df_insiders["Ticker"].tolist()

    try:
        yq = Ticker(tickers, asynchronous=True)
        # 'financial_data' contains the specific recommendation scores
        fin_data = yq.financial_data

        analyst_data = []
        for t in tickers:
            # Check if we got valid data for this ticker
            if isinstance(fin_data, dict) and t in fin_data:
                data = fin_data[t]
                # Ensure it's a dictionary and has the key we need
                if isinstance(data, dict) and "recommendationMean" in data:
                    score = data.get("recommendationMean")

                    # Only keep valid scores (sometimes they are None)
                    if score is not None:
                        analyst_data.append(
                            {
                                "Ticker": t,
                                "Analyst_Score": score,
                                "Analyst_Verdict": data.get("recommendationKey", "N/A"),
                            }
                        )

        df_analyst = pd.DataFrame(analyst_data)

        if df_analyst.empty:
            print(" No Analyst ratings found for these tickers.")
            return df_insiders  # Return original if no data found

        # Merge with the Insider DataFrame
        merged = pd.merge(df_insiders, df_analyst, on="Ticker", how="inner")

        # FILTER: Keep only scores <= max_score (Lower is better)
        final_df = merged[merged["Analyst_Score"] <= max_score].copy()

        print(
            f"  Analyst Filter: {len(merged)} -> {len(final_df)} stocks (Min Rating: Buy)."
        )
        return final_df.sort_values(by="Analyst_Score", ascending=True)

    except Exception as e:
        print(f"  Error in Analyst Filter: {e}")
        return df_insiders


# ==========================================
# 3. EXECUTION PIPELINE
# ==========================================

# A. Setup Tickers
if "fortress_df" in locals() and not fortress_df.empty:
    target_tickers = fortress_df["Ticker"].tolist()
else:
    # Backup list just in case need to update periodically
    target_tickers = [
        "PET.TO",
        "MFI.TO",
        "TXG.TO",
        "SAP.TO",
        "PAAS.TO",
        "NEO.TO",
        "WPM.TO",
        "FNV.TO",
        "LUG.TO",
        "DPM.TO",
        "ASM.TO",
        "PNG.V",
        "DSG.TO",
        "KNT.TO",
        "GGD.TO",
        "GRGD.TO",
        "WDO.TO",
        "OGC.TO",
        "DNG.TO",
        "CLS.TO",
    ]

# B. Run Insider Filter
insider_winners = filter_for_insider_buying(target_tickers)

# C. Run Analyst Filter (NEW STEP)
# We overwrite 'Fortress_insiders' so it works with Data Wrangler
if not insider_winners.empty:
    Fortress_insiders_Analyst_buy = filter_for_analyst_ratings(
        insider_winners, max_score=2.5
    )
else:
    Fortress_insiders_Analyst_buy = pd.DataFrame()

# D. Display Result
if not Fortress_insiders_Analyst_buy.empty:
    print(
        f"\n  Final List: {len(Fortress_insiders_Analyst_buy)} stocks (Fortress + Insider Buying + Analyst Buy Rating)"
    )
    Fortress_insiders_Analyst_buy.head()
else:
    print("No stocks passed all filters.")

 Scanning 220 stocks for Insider Buying...
  Analyst Filter: 12 -> 10 stocks (Min Rating: Buy).

  Final List: 10 stocks (Fortress + Insider Buying + Analyst Buy Rating)


In [30]:
# =============================================================================
# CELL 11: BURRY EV/EBITDA FILTER For Fortress Stocks
# =============================================================================
# EV/EBITDA is a valuation metric popular with hedge fund manager Michael Burry.
#
# EV = Enterprise Value = Market Cap + Debt - Cash
#      (What it would cost to buy the whole company)
#
# EBITDA = Earnings Before Interest, Taxes, Depreciation, and Amortization
#          (Proxy for operating cash flow)
#
# EV/EBITDA tells you how many years of cash flow it would take to buy the company.
# Lower is better (cheaper stock).
# We compare each stock to its SECTOR AVERAGE to find relative value.


def filter_burry_ev_ebitda(df_input):
    """
    Find stocks trading at a discount to their sector average.

    Logic: A stock with EV/EBITDA of 8x is "cheap" in a sector
           where the average is 15x.

    Args:
        df_input: DataFrame of stocks to analyze (usually fortress_df)

    Returns:
        DataFrame: Stocks cheaper than their sector average
    """
    if df_input is None or df_input.empty:
        print(" Input DataFrame is empty.")
        return pd.DataFrame()

    print(f"Analyzing EV/EBITDA for {len(df_input)} Fortress stocks...")

    tickers = df_input["Ticker"].tolist()

    # Fetch data in bulk
    try:
        yq = Ticker(tickers, asynchronous=True)
        data = yq.get_modules("defaultKeyStatistics financialData summaryDetail")
    except Exception as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()

    ev_data = []

    for ticker in tickers:
        try:
            ticker_data = data.get(ticker, {})
            if isinstance(ticker_data, str):
                continue

            stats = ticker_data.get("defaultKeyStatistics", {})
            fin_data = ticker_data.get("financialData", {})
            summary = ticker_data.get("summaryDetail", {})

            # Try to get pre-calculated EV/EBITDA
            ev_ebitda = stats.get("enterpriseToEbitda")

            # If not available, calculate manually
            if ev_ebitda is None:
                try:
                    market_cap = summary.get("marketCap")
                    total_debt = fin_data.get("totalDebt")
                    total_cash = fin_data.get("totalCash")
                    ebitda = fin_data.get("ebitda")

                    if all(
                        v is not None
                        for v in [market_cap, total_debt, total_cash, ebitda]
                    ):
                        if ebitda != 0:
                            # EV = Market Cap + Debt - Cash
                            enterprise_value = market_cap + total_debt - total_cash
                            ev_ebitda = enterprise_value / ebitda
                except:
                    print(f"Error calculating EV/EBITDA for {ticker}.")
                    pass

            # Only include positive values (profitable companies)
            if ev_ebitda is not None and ev_ebitda > 0:
                ev_data.append({"Ticker": ticker, "EV/EBITDA": round(ev_ebitda, 2)})
        except:
            print(f"Error processing {ticker}.")
            continue

    df_vals = pd.DataFrame(ev_data)

    if df_vals.empty:
        print("Could not retrieve EV/EBITDA data.")
        return pd.DataFrame()

    # Merge with sector data
    merged_df = pd.merge(df_input, df_vals, on="Ticker", how="inner")

    # Calculate sector averages
    print("\n--- SECTOR AVERAGES (EV/EBITDA) ---")
    sector_stats = merged_df.groupby("Sector")["EV/EBITDA"].mean().reset_index()
    sector_stats.rename(columns={"EV/EBITDA": "Sector_Avg_EV_EBITDA"}, inplace=True)
    sector_stats["Sector_Avg_EV_EBITDA"] = sector_stats["Sector_Avg_EV_EBITDA"].round(2)

    print(sector_stats.to_string(index=False))

    # Merge with sector averages
    final_df = pd.merge(merged_df, sector_stats, on="Sector", how="left")

    # Filter: Keep only stocks CHEAPER than sector average
    burry_picks = final_df[
        final_df["EV/EBITDA"] < final_df["Sector_Avg_EV_EBITDA"]
    ].copy()

    # Calculate discount percentage
    # Discount = 1 - (Stock EV/EBITDA / Sector Average)
    burry_picks["Discount_%"] = round(
        (1 - (burry_picks["EV/EBITDA"] / burry_picks["Sector_Avg_EV_EBITDA"])) * 100, 2
    )

    # Sort by biggest discount
    burry_picks = burry_picks.sort_values(by="Discount_%", ascending=False)

    return burry_picks


# Run the Burry filter
if "fortress_df" in locals() and not fortress_df.empty:
    Fortress_Burry_EV_EBITDA = filter_burry_ev_ebitda(fortress_df)

    if not Fortress_Burry_EV_EBITDA.empty:
        print(f"\nFound {len(Fortress_Burry_EV_EBITDA)} Undervalued Stocks")
        Fortress_Burry_EV_EBITDA.head()

Analyzing EV/EBITDA for 220 Fortress stocks...

--- SECTOR AVERAGES (EV/EBITDA) ---
                Sector  Sector_Avg_EV_EBITDA
       Basic Materials                 15.97
Communication Services                 19.16
     Consumer Cyclical                 14.37
    Consumer Defensive                 15.98
                Energy                  9.27
            Healthcare                 20.85
           Industrials                 20.93
            Technology                 22.16

Found 129 Undervalued Stocks


In [14]:
# =============================================================================
# CELL 11a: BURRY TECHNICAL SCAN (TREND + SQUEEZE)
# =============================================================================
# This combines two strategies into one efficient pass:
# 1. SAFETY: Filters out downtrends (Price < 200 SMA).
# 2. OPPORTUNITY: Calculates "Volatility Squeeze" to find coiling stocks.

import yfinance as yf
import pandas as pd


def apply_burry_technicals(df_input):
    """
    Fetches 1y history ONCE.
    Applies Trend Filter (Price > SMA200).
    Calculates Squeeze Metrics (Bandwidth).
    """
    if df_input is None or df_input.empty:
        print("   [Burry Technicals] Input DataFrame is empty. Skipping.")
        return pd.DataFrame()

    tickers = df_input["Ticker"].tolist()
    print(f"\n--- BURRY TECHNICALS: Analyzing {len(tickers)} candidates ---")

    # 1. Fetch History (1 Year is enough for both SMA200 and Squeeze)
    try:
        data = yf.download(
            tickers,
            period="1y",
            interval="1d",
            progress=False,
            group_by="ticker",
            auto_adjust=True,
        )
    except Exception as e:
        print(f"   [Error] Failed to fetch history: {e}")
        return pd.DataFrame()

    results_data = []

    for ticker in tickers:
        try:
            # Handle data structure
            if len(tickers) > 1:
                if ticker not in data.columns.levels[0]:
                    continue
                df_hist = data[ticker].copy()
            else:
                df_hist = data.copy()

            # Need sufficient data
            if len(df_hist) < 200:
                print(f"   [Skipping] {ticker}: Insufficient history (<200 days).")
                continue

            # --- METRIC 1: TREND (SMA 200) ---
            sma_200 = df_hist["Close"].rolling(window=200).mean().iloc[-1]
            current_price = df_hist["Close"].iloc[-1]

            # HARD FILTER: Must be in Uptrend
            if current_price < sma_200:
                continue

            dist_sma_pct = ((current_price - sma_200) / sma_200) * 100

            # --- METRIC 2: SQUEEZE (Bollinger Bands) ---
            sma_20 = df_hist["Close"].rolling(window=20).mean()
            std_20 = df_hist["Close"].rolling(window=20).std()

            upper = sma_20 + (2 * std_20)
            lower = sma_20 - (2 * std_20)
            bandwidth = (upper - lower) / sma_20

            # Check Squeeze Status (Lowest 15% of last 6 months)
            recent_bw = bandwidth.tail(126)  # ~6 months
            current_bw = recent_bw.iloc[-1]
            squeeze_threshold = recent_bw.quantile(0.15)
            is_squeezing = current_bw <= squeeze_threshold

            # --- COMPILE RESULT ---
            # Get base fundamental data
            base_row = df_input[df_input["Ticker"] == ticker].iloc[0].to_dict()

            # Add Technical Data
            base_row["Price"] = round(current_price, 2)
            base_row["SMA_200"] = round(sma_200, 2)
            base_row["Trend_Dist_%"] = round(dist_sma_pct, 2)
            base_row["Bandwidth_%"] = round(current_bw * 100, 2)
            base_row["In_Squeeze"] = "YES" if is_squeezing else "No"

            results_data.append(base_row)

        except Exception as e:
            print(f"   [Error] Processing {ticker}: {e}")
            continue

    return pd.DataFrame(results_data)


# =========================================
# EXECUTION
# =========================================

if "Fortress_Burry_EV_EBITDA" in locals() and not Fortress_Burry_EV_EBITDA.empty:

    # Run the consolidated scan
    Burry_Technicals_DF = apply_burry_technicals(Fortress_Burry_EV_EBITDA)

    if not Burry_Technicals_DF.empty:
        # Sort by "Squeeziness" (Bandwidth), but favor those in Squeeze first
        # We sort by In_Squeeze (descending, so YES comes first) then Bandwidth (ascending)
        Burry_Technicals_DF = Burry_Technicals_DF.sort_values(
            by=["In_Squeeze", "Bandwidth_%"], ascending=[False, True]
        )

        # Save results
        TECH_CSV = "YfinanceDataDump/Burry_Technical_Picks_USA.csv"
        Burry_Technicals_DF.to_csv(TECH_CSV, index=False)

        print(f"\nSUCCESS: Technical picks saved to '{TECH_CSV}'")
        print(f"Original Undervalued Count: {len(Fortress_Burry_EV_EBITDA)}")
        print(f"Survivors (Uptrend Only): {len(Burry_Technicals_DF)}")


--- BURRY TECHNICALS: Analyzing 129 candidates ---

SUCCESS: Technical picks saved to 'YfinanceDataDump/Burry_Technical_Picks_USA.csv'
Original Undervalued Count: 129
Survivors (Uptrend Only): 96


In [31]:
# ==========================================
# 12. ANALYST FILTER FUNCTION FOR Burry EV/EBITDA PICKS (NEW)
# ==========================================
def filter_for_analyst_ratings(Fortress_Burry_EV_EBITDA, max_score=2.5):
    """
    Fetches analyst data for the insider winners and filters for 'Buy' or better.
    Scale: 1.0 = Strong Buy, 5.0 = Sell.
    Cutoff: 2.5 ensures we get 'Buy' and 'Strong Buy'.
    """
    if Fortress_Burry_EV_EBITDA.empty:
        return Fortress_Burry_EV_EBITDA

    tickers = Fortress_Burry_EV_EBITDA["Ticker"].tolist()

    try:
        yq = Ticker(tickers, asynchronous=True)
        # 'financial_data' contains the specific recommendation scores
        fin_data = yq.financial_data

        analyst_data = []
        for t in tickers:
            # Check if we got valid data for this ticker
            if isinstance(fin_data, dict) and t in fin_data:
                data = fin_data[t]
                # Ensure it's a dictionary and has the key we need
                if isinstance(data, dict) and "recommendationMean" in data:
                    score = data.get("recommendationMean")

                    # Only keep valid scores (sometimes they are None)
                    if score is not None:
                        analyst_data.append(
                            {
                                "Ticker": t,
                                "Analyst_Score": score,
                                "Analyst_Verdict": data.get("recommendationKey", "N/A"),
                            }
                        )

        df_analyst = pd.DataFrame(analyst_data)

        if df_analyst.empty:
            print("No Analyst ratings found for these tickers.")
            return Fortress_Burry_EV_EBITDA  # Return original if no data found

        # Merge with the Fortress DataFrame
        merged = pd.merge(
            Fortress_Burry_EV_EBITDA, df_analyst, on="Ticker", how="inner"
        )

        # FILTER: Keep only scores <= max_score (Lower is better)
        final_df = merged[merged["Analyst_Score"] <= max_score].copy()

        print(
            f"Analyst Filter: {len(merged)} -> {len(final_df)} stocks (Min Rating: Buy)."
        )
        return final_df.sort_values(by="Analyst_Score", ascending=True)

    except Exception as e:
        print(f"Error in Analyst Filter: {e}")
        return Fortress_Burry_EV_EBITDA


# ==========================================
# 3. EXECUTION PIPELINE
# ==========================================

# A. Setup Tickers
if "Fortress_Burry_EV_EBITDA" in locals() and not Fortress_Burry_EV_EBITDA.empty:
    target_tickers = Fortress_Burry_EV_EBITDA["Ticker"].tolist()


# B. Run Insider Filter
Fortress_Burry_EV_EBITDA = filter_burry_ev_ebitda(fortress_df)

# C. Run Analyst Filter (NEW STEP)
# We overwrite 'Fortress_insiders' so it works with Data Wrangler
if not Fortress_Burry_EV_EBITDA.empty:
    Fortress_Burry_Analyst_buy = filter_for_analyst_ratings(
        Fortress_Burry_EV_EBITDA, max_score=2.5
    )
else:
    Fortress_Burry_Analyst_buy = pd.DataFrame()

# D. Display Result
if not Fortress_Burry_Analyst_buy.empty:
    print(
        f"\nFinal List: {len(Fortress_Burry_Analyst_buy)} stocks (Fortress + Burry + Analyst Buy Rating)"
    )
    Fortress_Burry_Analyst_buy.head()
else:
    print("No stocks passed all filters.")

Analyzing EV/EBITDA for 220 Fortress stocks...

--- SECTOR AVERAGES (EV/EBITDA) ---
                Sector  Sector_Avg_EV_EBITDA
       Basic Materials                 15.97
Communication Services                 19.16
     Consumer Cyclical                 14.37
    Consumer Defensive                 15.98
                Energy                  9.27
            Healthcare                 20.85
           Industrials                 20.93
            Technology                 22.16
Analyst Filter: 125 -> 104 stocks (Min Rating: Buy).

Final List: 104 stocks (Fortress + Burry + Analyst Buy Rating)


In [33]:
# ==========================================
# 13. THE "DEEP VALUE" INTERSECTION (Buffett + Burry)
# ==========================================

# 1. Ensure we have the Buffett Data
if "Buffett_Value_DF" not in locals():
    print("Buffett Data not found. Running scan now...")
    if "get_buffett_value_picks" in globals() and "final_results" in locals():
        Buffett_Value_DF = get_buffett_value_picks(final_results)
    else:
        print("Missing 'final_results' or 'get_buffett_value_picks' function.")
        Buffett_Value_DF = pd.DataFrame()

# 2. Ensure we have the Burry Data
if "Fortress_Burry_EV_EBITDA" not in locals():
    print("Please run the Burry EV/EBITDA filter cell first.")
    Fortress_Burry_EV_EBITDA = pd.DataFrame()

# 3. THE MERGE (Finding the Overlap)
if not Buffett_Value_DF.empty and not Fortress_Burry_EV_EBITDA.empty:

    # Merge on Ticker to find stocks that appear in BOTH lists
    # We use an 'inner' join, which means "keep only if in both"
    Deep_Value_Intersection = pd.merge(
        Buffett_Value_DF[["Ticker", "P/B Ratio", "ROE %", "Debt/Eq %"]],
        Fortress_Burry_EV_EBITDA[
            [
                "Ticker",
                "Price",
                "Sector",
                "EV/EBITDA",
                "Sector_Avg_EV_EBITDA",
                "Discount_%",
                "Tier",
            ]
        ],
        on="Ticker",
        how="inner",
    )

    if not Deep_Value_Intersection.empty:
        print("\n" + "=" * 60)
        print(f"DEEP VALUE GEMS FOUND: {len(Deep_Value_Intersection)}")
        print("=" * 60)
        print(
            "Criteria: Trading < Book Value (Buffett) AND Cheaper than Sector (Burry)"
        )

        # Sort by the "Discount" (how cheap they are vs sector)
        Deep_Value_Intersection = Deep_Value_Intersection.sort_values(
            by="Discount_%", ascending=False
        )

        cols = [
            "Ticker",
            "Price",
            "Tier",
            "P/B Ratio",
            "EV/EBITDA",
            "Sector_Avg_EV_EBITDA",
            "Discount_%",
            "Sector",
        ]
        Deep_Value_Intersection[cols].head(5)

    else:
        print("\n No stocks passed BOTH filters.")
        print(
            "This means no stock is both 'Below Book Value' AND 'Cheaper than Sector Average' at the same time."
        )
        print(
            f"Buffett Count: {len(Buffett_Value_DF)} | Burry Count: {len(Fortress_Burry_EV_EBITDA)}"
        )

else:
    print("Cannot combine. One of the filters returned 0 results.")


DEEP VALUE GEMS FOUND: 2
Criteria: Trading < Book Value (Buffett) AND Cheaper than Sector (Burry)


In [35]:
# =============================================================================
# CELL 14: THE "HIGH CONVICTION" INTERSECTION (BURRY + INSIDERS)
# =============================================================================
# This finds stocks that are BOTH:
# 1. Undervalued vs their Sector (Burry EV/EBITDA)
# 2. Being accumulated by Insiders (Insider Confidence)
#
# This is a powerful "Anti-Value Trap" filter. Insiders rarely buy
# sinking ships, even if they look cheap on paper.

# --- 1. DETECT BEST AVAILABLE DATAFRAMES ---

# Try to find the Insider list (preferring the one with Analyst ratings if available)
if (
    "Fortress_insiders_Analyst_buy" in locals()
    and not Fortress_insiders_Analyst_buy.empty
):
    df_insider_source = Fortress_insiders_Analyst_buy
    print("Using Analyst-Vetted Insider List.")
elif "Fortress_insiders" in locals() and not Fortress_insiders.empty:
    df_insider_source = Fortress_insiders
    print("Using Raw Insider List (No Analyst Check).")
else:
    df_insider_source = pd.DataFrame()
    print("Warning: No Insider data found.")

# Try to find the Burry list (preferring the one with Analyst ratings if available)
if "Fortress_Burry_Analyst_buy" in locals() and not Fortress_Burry_Analyst_buy.empty:
    df_burry_source = Fortress_Burry_Analyst_buy
    print("Using Analyst-Vetted Burry List.")
elif "Fortress_Burry_EV_EBITDA" in locals() and not Fortress_Burry_EV_EBITDA.empty:
    df_burry_source = Fortress_Burry_EV_EBITDA
    print("Using Raw Burry List (No Analyst Check).")
else:
    df_burry_source = pd.DataFrame()
    print("Warning: No Burry EV/EBITDA data found.")

# --- 2. PERFORM THE INTERSECTION ---

if not df_insider_source.empty and not df_burry_source.empty:

    # We merge on 'Ticker'. 'inner' means keep only stocks found in BOTH lists.
    # We use suffixes=('', '_drop') to handle duplicate columns (like Price)
    high_conviction_df = pd.merge(
        df_burry_source[
            [
                "Ticker",
                "Price",
                "Sector",
                "Tier",
                "EV/EBITDA",
                "Sector_Avg_EV_EBITDA",
                "Discount_%",
            ]
        ],
        df_insider_source[["Ticker", "Insider_Buys_Count", "Net_Shares_Bought"]],
        on="Ticker",
        how="inner",
    )

    if not high_conviction_df.empty:
        # Sort by Discount % (Deepest value first)
        high_conviction_df = high_conviction_df.sort_values(
            by="Discount_%", ascending=False
        )

        print("\n" + "=" * 60)
        print(f"HIGH CONVICTION WINNERS FOUND: {len(high_conviction_df)}")
        print("=" * 60)
        print("Criteria: Cheap vs Sector (Burry) + Insider Buying")

        # Reorder columns for readability
        cols = [
            "Ticker",
            "Price",
            "Sector",
            "Tier",
            "Discount_%",
            "EV/EBITDA",
            "Insider_Buys_Count",
            "Net_Shares_Bought",
        ]

        # Display nicely
        high_conviction_df[cols].head()

        # Optional: Save to CSV
        save_path = os.path.join(DATA_FOLDER, "High_Conviction_Picks_USA.csv")
        high_conviction_df.to_csv(save_path, index=False)
        print(f"\nSaved to: {save_path}")

    else:
        print(
            "\nNo overlap found. No stock is both 'Cheap vs Sector' AND 'Insider Buy' target."
        )
        print(
            f"Burry Count: {len(df_burry_source)} | Insider Count: {len(df_insider_source)}"
        )
else:
    print("\nCannot run intersection. Missing one or both source lists.")

Using Analyst-Vetted Insider List.
Using Analyst-Vetted Burry List.

HIGH CONVICTION WINNERS FOUND: 8
Criteria: Cheap vs Sector (Burry) + Insider Buying

Saved to: YfinanceDataDump\High_Conviction_Picks_USA.csv


In [ ]:
# ==========================================
# Watchlist Combiner (Finviz + YFinance)
# ==========================================


import pandas as pd
import yfinance as yf
from finvizfinance.quote import finvizfinance
import time
import numpy as np

# --- 1. INPUT YOUR MANUAL LIST HERE ---
MY_TICKERS = [
    "SHLS",
    "BANC",
    "ONB",
    "UBER",
    "ADMA",
    "MIR",
    "APG",
    "SEI",
    "FLEX",
    "DD",
    "SVM",
    "GIL",
]


def get_combined_watchlist(ticker_list):
    print(f"--- Processing {len(ticker_list)} stocks ---")

    # --- PART A: Get Analyst Ratings from Finviz ---
    print("1. Fetching Analyst Ratings from Finviz...")
    finviz_data = []

    for ticker in ticker_list:
        try:
            stock = finvizfinance(ticker)
            info = stock.ticker_fundament()

            finviz_data.append(
                {
                    "Ticker": ticker,
                    "Recom": info.get("Recom", np.nan),
                    "Target_Price": info.get("Target Price", np.nan),
                }
            )
            time.sleep(0.5)

        except Exception as e:
            print(f"   Skipping Finviz for {ticker}: {e}")
            finviz_data.append(
                {"Ticker": ticker, "Recom": np.nan, "Target_Price": np.nan}
            )

    df_finviz = pd.DataFrame(finviz_data)

    # --- PART B: Get Real-Time Stats from yfinance ---
    print("2. Fetching Price & Volatility from yfinance...")

    try:
        # Download data (1 Year is perfect for 52-Week MA)
        data = yf.download(
            ticker_list,
            period="1y",
            interval="1d",
            group_by="ticker",
            progress=False,
            threads=True,
        )
        yf_stats = []

        for ticker in ticker_list:
            try:
                # --- FIXED: Robust Data Extraction ---
                if isinstance(data.columns, pd.MultiIndex):
                    if ticker in data.columns.levels[0]:
                        df = data[ticker].copy()
                    else:
                        print(f"   Warning: {ticker} not found in yfinance download.")
                        continue
                else:
                    df = data.copy()

                # Cleanup
                df = df.dropna(subset=["Close"])
                if len(df) < 20:
                    print(f"   Warning: Not enough data for {ticker}")
                    continue

                # --- MATH CALCULATIONS ---
                current_price = df["Close"].iloc[-1]
                prev_close = df["Close"].iloc[-2]

                high_52 = df["High"].max()
                drop_from_high = ((current_price - high_52) / high_52) * 100

                change_pct = ((current_price - prev_close) / prev_close) * 100

                # Volatility (30-day Std Dev)
                volatility = df["Close"].pct_change().std() * 100

                # Relative Volume
                curr_vol = df["Volume"].iloc[-1]
                avg_vol = df["Volume"].tail(30).mean()
                rel_vol = curr_vol / avg_vol if avg_vol > 0 else 0

                # --- NEW: 52-Week Moving Average ---
                # Since we fetched exactly 1 year ('1y'), the mean of the whole column is the 52W MA
                ma_52w = df["Close"].mean()

                # Distance from MA (Optional but helpful metric)
                # dist_ma = ((current_price - ma_52w) / ma_52w) * 100

                yf_stats.append(
                    {
                        "Ticker": ticker,
                        "Price": round(current_price, 2),
                        "Change_%": round(change_pct, 2),
                        "52W_MA": round(ma_52w, 2),  # <--- Added Here
                        "Drop_from_High_%": round(drop_from_high, 2),
                        "Volatility_%": round(volatility, 2),
                        "Rel_Volume": round(rel_vol, 2),
                    }
                )

            except Exception as e:
                print(f"   Error calculating stats for {ticker}: {e}")
                continue

        df_yf = pd.DataFrame(yf_stats)

    except Exception as e:
        print(f"yfinance Critical Error: {e}")
        return pd.DataFrame()

    # --- PART C: Merge ---
    if not df_finviz.empty:
        if not df_yf.empty:
            master_df = pd.merge(df_finviz, df_yf, on="Ticker", how="outer")
        else:
            master_df = df_finviz

        # Added '52W_MA' to this list so it displays in the final table
        cols = [
            "Ticker",
            "Price",
            "Change_%",
            "52W_MA",
            "Drop_from_High_%",
            "Recom",
            "Target_Price",
            "Rel_Volume",
            "Volatility_%",
        ]

        final_cols = [c for c in cols if c in master_df.columns]
        return master_df[final_cols]
    else:
        return pd.DataFrame()


# --- RUN IT ---
watchlist_df = get_combined_watchlist(MY_TICKERS)

if not watchlist_df.empty:
    if "Drop_from_High_%" in watchlist_df.columns:
        watchlist_df["Drop_from_High_%"] = pd.to_numeric(
            watchlist_df["Drop_from_High_%"], errors="coerce"
        )
        print("\n--- Final Watchlist ---")
        watchlist_df.sort_values(by="Drop_from_High_%", ascending=True).head()
    else:
        watchlist_df.head()
else:
    print("No data found.")

--- Processing 12 stocks ---
1. Fetching Analyst Ratings from Finviz...
2. Fetching Price & Volatility from yfinance...

--- Final Watchlist ---


In [39]:
tickers_gemini = ["SHLS"]
import os
from google import genai
from google.genai import types
from IPython.display import display, Markdown

# ==========================================
# Senior Analyst with INTERACTIVE FOLLOW-UPS
# ==========================================

KEY_FILE_PATH = (
    r"C:\Users\jdcc3\OneDrive - McMaster University\Gemini API Key\gemini_key.txt"
)


def load_api_key(filepath):
    try:
        with open(filepath, "r") as f:
            return f.read().strip()
    except Exception as e:
        print(f"Error reading key file: {e}")
        return None


api_key = load_api_key(KEY_FILE_PATH)

if api_key:
    os.environ["GEMINI_API_KEY"] = api_key
    print("API Key loaded securely.")
else:
    print("CRITICAL: API Key not loaded.")


# ==========================================
# INTERACTIVE ANALYSIS FUNCTION
# ==========================================
def analyze_and_chat_gemini_3(tickers_gemini, company_name=None):

    if not os.environ.get("GEMINI_API_KEY"):
        print("Stop: No API Key found.")
        return

    print(
        f"\nGemini 3 is thinking (High Reasoning Mode)... analyzing ${tickers_gemini}..."
    )

    # 1. Initialize Client
    client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

    # 2. Configure the Chat with Tools (Search) and Reasoning
    config = types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            include_thoughts=False, thinking_level="HIGH"
        ),
        tools=[types.Tool(google_search=types.GoogleSearch())],
        response_modalities=["TEXT"],
    )

    # 3. Create the Chat Session (This holds the history)
    chat = client.chats.create(
        model="gemini-3-pro-preview", config=config  # Recommended for reasoning + speed
    )

    # 4. Define the Initial Prompt
    initial_prompt = f"""
    You are a Senior Equity Research Analyst using the Gemini 3 Reasoning Engine. 
    Perform a deep "Market Sentiment Analysis" on {tickers_gemini}.
    
    Step 1: SEARCH. Use Google Search to find the latest (last 30 days) news, analyst notes, and SEC filings.
    Step 2: REASON. Analyze the search results to determine the true market psychology.
    
    Investigate these 4 Pillars:
    1. **News Virality**: Are headlines fear-mongering or euphoric?
    2. **Analyst Shifts**: Are price targets moving UP or DOWN?
    3. **Institutional Flows**: Hedge funds or insiders buying/selling?
    4. **The "Whisper" Number**: What are traders saying on forums?

    **OUTPUT FORMAT:**
    Produce a professional Markdown report:
    
    ## Gemini Sentiment Report: {tickers_gemini}
    **Reasoning Depth:** High
    **Sentiment Score:** [1-10]
    **Verdict:** [Buy / Hold / Sell / Speculative]
    
    ### 1. The Bull Thesis
    * ...
    
    ### 2. The Bear Thesis
    * ...
    
    ### 3. Deep Dive Analysis
    * **News Analysis**: ...
    * **Smart Money**: ...
    * **Financial Statement Analysis**: (Historic performance last 3y + outlook)
    
    ### 4. Conclusion
    """

    try:
        # 5. Send Initial Request
        response = chat.send_message(initial_prompt)
        display(Markdown(response.text))

        # ==========================================
        # NEW: INTERACTIVE LOOP
        # ==========================================
        print("\n" + "=" * 50)
        print(f" INTERACTIVE SESSION: Ask follow-ups about {tickers_gemini}")
        print(" Type 'exit' or 'quit' to stop.")
        print("=" * 50 + "\n")

        while True:
            # Get user input
            user_question = input("\nYour Question type stop to close this prompt: ")

            # Check for exit condition
            if user_question.lower() in ["exit", "quit", "stop"]:
                print("Ending session.")
                break

            # Send follow-up to the SAME chat session
            print("Analyzing...")
            follow_up_response = chat.send_message(user_question)
            display(Markdown(follow_up_response.text))

    except Exception as e:
        print(f"Error: {e}")


# ==========================================
# EXECUTION
# ==========================================
if os.environ.get("GEMINI_API_KEY"):
    analyze_and_chat_gemini_3(tickers_gemini, tickers_gemini)

API Key loaded securely.

Gemini 3 is thinking (High Reasoning Mode)... analyzing $['SHLS']...


The following Market Sentiment Analysis for **Shoals Technologies Group, Inc. (SHLS)** evaluates the company's standing as of **January 20, 2026**, incorporating the latest search data from the last 30 days.

## Gemini Sentiment Report: ['SHLS']
**Reasoning Depth:** High
**Sentiment Score:** 6.5/10
**Verdict:** **Speculative Buy**

### 1. The Bull Thesis
*   **Record Backlog & Growth:** As of Q3 2025, SHLS reported a record backlog and awarded orders of **$720.9 million** (up 21% YoY). This provides high revenue visibility for 2026 despite earlier headwinds.
*   **Sector Tailwind:** In early January 2026, the solar sector experienced a broad "risk-on" rotation, lifting peers like Canadian Solar (CSIQ) and Enphase (ENPH). SHLS rallied **~11-12%** in sympathy, suggesting it is a high-beta beneficiary of improving macro sentiment (likely tied to interest rate expectations).
*   **Institutional Accumulation:** Recent 13F filings show significant accumulation by "smart money." Voya Investment Management increased its position by nearly **2,000%** and Swedbank by **200%** in late 2025, signaling institutional confidence in a turnaround.
*   **IP Reinforcement:** Despite losing the initial ITC battle in early 2025, SHLS has aggressively filed complaints using *new* patents ('375 and '376) to rebuild its legal moat against competitors like Voltage, LLC.

### 2. The Bear Thesis
*   **The "Broken Moat" Risk:** The major bearish catalyst remains the **ITC's reversal in January 2025**, which found "no violation" by competitor Voltage, LLC regarding the key '153 patent. This legally allows competitors to market similar "trunk bus" solutions, effectively piercing SHLS's monopoly-like patent protection until new litigation succeeds.
*   **Insider Selling:** Contrary to retail "whispers" of insider buying, actual SEC filings from **December 2025** show *selling*. CFO Dominic Bardos and President Jeffery Tolnar sold shares, which often dampens momentum and contradicts the "undervalued" narrative.
*   **Execution Misses:** The company missed EPS estimates in Q3 2025 ($0.07 vs. $0.13 expected), creating a "show me" story where investors demand perfect execution on margins before re-rating the stock higher.

### 3. Deep Dive Analysis

#### **News Virality & Psychology**
*   **Current Mood:** **Cautiously Optimistic.** The headlines in Jan 2026 are dominated by price action ("Trading 11.2% Higher") rather than fundamental news. This indicates a "relief rally" or short-covering event driven by macro factors rather than a specific company catalyst.
*   **The Narrative Shift:** The conversation has shifted from "Litigation Panic" (early 2025) to "Backlog Execution" (2026). The market seems to be pricing in the competitive threat from Voltage but betting that the total addressable market (TAM) expansion will lift all boats.

#### **Smart Money & Institutional Flows**
*   **Institutions:** **Net Buyers.** The massive position increases by Voya and Swedbank suggest that funds viewed the 2025 dip as a buying opportunity. The "Fund Sentiment Score" remains high.
*   **Insiders:** **Net Sellers.** The recent sales in Dec 2025 by top executives (CFO, President) are a red flag. While these may be tax-related or routine, they signal that management is not aggressively buying the dip at these levels.

#### **Litigation Update (Crucial Context)**
*   **Status:** **Defensive.**
    *   **Jan 2025:** ITC rules *against* SHLS (finding no violation by Voltage).
    *   **Feb 2025:** SHLS appeals to the Federal Circuit.
    *   **Current (Jan 2026):** SHLS is relying on *newly issued* patents to file fresh complaints. Investors must understand that the original "impenetrable" IP moat is currently compromised, making the stock more dependent on execution than patent protection.

#### **The "Whisper" Number**
*   **Retail Sentiment:** **Bullish.** Forums (Stocktwits, Reddit) are circulating a "catch-up trade" narrative, targeting **$12-$15**. Traders are focused on the high short interest and the stock's lag behind peers.
*   **Consensus Disconnect:** Retail traders are ignoring the insider selling, focusing instead on the "buyback" narrative (which is older news).

### 4. Conclusion
Shoals Technologies Group (SHLS) is a **high-risk, high-reward play** for early 2026. The company is successfully growing its order book and expanding internationally (e.g., Chile), which supports the **Bull Case**. However, the **Bear Case** is structural: the loss of its absolute patent monopoly means it must now compete on price and efficiency rather than legal exclusivity.

**Verdict:** The stock is a **Speculative Buy** for investors betting on a solar sector recovery. The heavy institutional buying provides a floor, but the recent insider selling and legal overhang suggest upside may be capped until the company delivers a "beat-and-raise" quarter to prove its competitive durability.


 INTERACTIVE SESSION: Ask follow-ups about ['SHLS']
 Type 'exit' or 'quit' to stop.

Ending session.
